In [18]:
!pip install pymongo
!pip install flatten_dict

import pymongo
import requests
import json
import time

from flatten_dict import flatten

In [19]:
## Get account details by providing the account name
def requestMatchData(match_id, APIKey):
    URL = "https://euw1.api.riotgames.com/lol/match/v4/matches/" + match_id + "?api_key=" + APIKey
    response = requests.get(URL)
    return response.json()

In [20]:
## Get account details by providing the account name
def requestSummonerData(summonerName, APIKey):
    URL = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKey
    response = requests.get(URL)
    return response.json()

In [21]:
## Get an account's ranked match data by account ID
def requestRankedData(ID, APIKey):
    URL = "https://euw1.api.riotgames.com/lol/league/v4/positions/by-summoner/" + str(ID) + "?api_key=" + APIKey
    response = requests.get(URL)
    return response.json()

In [22]:
def requestMatchesByAccountId(accountId, APIKey):
    """
    Get matchlist for games played on given account ID
    and platform ID and filtered using given filter 
    parameters, if any
    """
    URL = "https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + str(accountId) + "?api_key=" + APIKey
    response = requests.get(URL)
    return response.json()

In [23]:
def getSummonersDataPropByName(sampleSummonerNames, target, APIKey):
    result=[]
    for summonerName in sampleSummonerNames:
        try:
            s=requestSummonerData(summonerName, APIKey)
            result.append(s[target])
        except:
            pass
    return result

In [24]:
def getMatchesByAccountId(summoners_account_id, APIKey):
    matchesByAccountId=[]
    for summoners_account_id in summoners_account_id:
        try:
            m=requestMatchesByAccountId(summoners_account_id, APIKey)
            matchesByAccountId.append(m.get('matches'))
        except:
            pass
    return matchesByAccountId

In [25]:
def getParticipantIdentitiesProp(participantIdentities, target):
    result=[]
    for i, partecipant in enumerate(participantIdentities):
        s=participantIdentities[i]['player'][target]
        result.append(s)
    return result

In [26]:
def getDataForCurrentMatch(partecipant_summoner_matches_list, g):
    info=[]
    # loop over the list of summoners
    for i, match in enumerate(partecipant_summoner_matches_list):
        # loop on each instance
        for j, data in enumerate(match):
            if data.get('gameId')==g:
                info.append(data)
                #print(f"{i}-{j} =====> \n {data}")
    return info

In [27]:
def _argumentPositional(array_dict, pos):
    """extract info in list as positional argument"""
    for i, element in enumerate(array_dict):
        return list([v for v in [list(array_dict[i].values())[pos] for i in range(len(array_dict))]])

In [28]:
# save object to a file
# HIGHEST_PROTOCOL for binary format
# 0 for text output
import pickle

def save_obj(obj, name, path_to_save):
    with open(path_to_save + '/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, 0)

In [29]:
def retrieveChampions(champion_list):
    
    # dummy champion global list TODO replace with API
    with open('../dummy/champion.json') as json_file:
        data = json.load(json_file).get('data')
    
    champion_test_list=[]
    for k in data.keys():
            champion_test_list.append(data[k])
            
    result={}
    for i, champion in enumerate(champion_test_list):    
        key=champion_test_list[i].get('key')
        if int(key) in champion_list:
            result[int(key)]=(champion)
    return result

In [30]:
def getOrderedExpandedData(champion_list):
    expData=retrieveChampions(champion_list)

    # sort dictionary
    index_map = {v: i for i, v in enumerate(champion_list)}
    return sorted(expData.items(), key=lambda pair: index_map[pair[0]])

In [31]:
def enrichParticipantIdentities(expData, participantIdentities):
    for i, participant in enumerate(participantIdentities):
        player=participant.get('player')
        player['hero']=expData[i][1]
        participant['player']=player
    return participantIdentities

In [32]:
# sample summoner names
sample_summoner_names = "Fnatic;Huni;Hide on bush;lanekingdom1;bbbbbbbbbbbbj;Doublelift".split(';')

In [33]:
### API KEYS ###
APIKey = "RGAPI-659679ed-037c-4e2c-843e-e4177d1b1626"

In [34]:
### DUMMY ###
def getSampleGameId(sample_summoner_names, APIKey):
    """Dummy function to extract a GameId list from Summoner names"""
    summoners_account_ids=getSummonersDataPropByName(sample_summoner_names, 'accountId', APIKey)
    matchesByAccountId= getMatchesByAccountId(summoners_account_ids, APIKey)

    gameIds=[]
    for i, d in enumerate(matchesByAccountId):
        for j in range(len(d)):
            gameIds.append(d[j]['gameId'])
    return gameIds

In [43]:
def main():
    
    # get sample gameId
    gameIds=getSampleGameId(sample_summoner_names, APIKey)
    
    
    game_dict={}
    for gameId in gameIds:

        # retrieve game data for each game id:
        gameData=requestMatchData(str(gameId), APIKey)
        participantIdentities=gameData['participantIdentities']

        # loop on array documents extract summoner names
        partecipant_summoner_name_list=getParticipantIdentitiesProp(participantIdentities, 'summonerName')

        # loop on array documents extract summoner data by name
        partecipant_summoner_account_id_list=getSummonersDataPropByName(partecipant_summoner_name_list, 'accountId', APIKey)

        # retrieve data by current summoner by each gameId
        partecipant_summoner_matches_list=getMatchesByAccountId(partecipant_summoner_account_id_list,APIKey)

        # summoner info for current match
        summoner_info_current_match=getDataForCurrentMatch(partecipant_summoner_matches_list, gameId)

        # champion list   
        champion_list=_argumentPositional(summoner_info_current_match, 2)

        #expanded info for players
        expData=getOrderedExpandedData(champion_list)

        # get enriched data
        participantIdentitiesEnriched=enrichParticipantIdentities(expData, participantIdentities)
        
        # get dict to write and store DataFrame
        game_dict[gameId]=participantIdentitiesEnriched
        
        # store as pickle
        save_obj(game_dict, f"{gameId}", 'games_data')
        
        
        break
    
    return game_dict

In [44]:
main()

{4318538512: [{'participantId': 1,
   'player': {'platformId': 'EUW1',
    'accountId': 'EykVQycJufwQyRq_DDnOVusu0APRvF62rPhd2ZEeOJbjwjw',
    'summonerName': 'DeltaOxytar',
    'summonerId': 'G1gKlTSYf3CsYeIsmLbeOxAtT-b_UiKsz9JfjYZPaAas9-Q',
    'currentPlatformId': 'EUW1',
    'currentAccountId': 'EykVQycJufwQyRq_DDnOVusu0APRvF62rPhd2ZEeOJbjwjw',
    'matchHistoryUri': '/v1/stats/player_history/EUW1/217018127',
    'profileIcon': 4291,
    'hero': {'version': '9.23.1',
     'id': 'Vladimir',
     'key': '8',
     'name': 'Vladimir',
     'title': 'the Crimson Reaper',
     'blurb': "A fiend with a thirst for mortal blood, Vladimir has influenced the affairs of Noxus since the empire's earliest days. In addition to unnaturally extending his life, his mastery of hemomancy allows him to control the minds and bodies of others as easily...",
     'info': {'attack': 2, 'defense': 6, 'magic': 8, 'difficulty': 7},
     'image': {'full': 'Vladimir.png',
      'sprite': 'champion4.png',
      